<a href="https://colab.research.google.com/github/DonJM/FCLB-TFOD/blob/master/FCLB_TFOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

#import neccessary python modules and libraries
import os
import shutil
from PIL import ImageEnhance
from PIL import Image
import uuid
import tarfile
import wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=e7ecc3176244c03d5a880b54bba7972416c167dbd1796f6963cd3150beed972a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


# Adding Classes for Path Setup and Project Dataset

In [4]:
class Setup_Path: 

    LABELS = ["fresh", "rotten"]

    def getFiles(self):
      files = {
          "LABELMAP":os.path.join(self.getWorkspaces()['ANNOTATIONS_PATH'], "label_map.pbtxt"),
          'TFRECORD_SCRIPT':os.path.join(self.getWorkspaces()['PREPROCESSING_PATH'], 'generate_tfrecord.py')
      }

      return files

    def getWorkspaces(self):
        paths = {
            "ADDONS_PATH":os.path.join("Tensorflow", "addons"),
            "LABEL_IMG_PATH":os.path.join("Tensorflow", "addons", "labelImg"),
            "TFOD_API_PATH":"Tensorflow",
            "WORKSPACE_PATH":os.path.join("Tensorflow", "workspace"),
            "TRAINING_DEMO_PATH":os.path.join("Tensorflow", "workspace", "training_demo"),
            "ANNOTATIONS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "annotations"),
            "EXPORTED_MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "exported-models"),
            "IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images"),
            "TRAINING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "train"),
            "TESTING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "test"),
            "MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo"),
            "PRETRAINED_MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "pre-trained-models"),
            "PREPROCESSED_IMG":os.path.join("Tensorflow", "addons", "preprocessedImg"),
            "ORIGINAL_IMGS":os.path.join("Tensorflow", "addons", "originalImg"),   
            "PREPROCESSING_PATH":os.path.join("Tensorflow", "scripts", "preprocessing"),
            "PROTOS_PATH":os.path.join("Tensorflow", "Google-Protobuf"),
        }

        return paths

    # def set_main_paths(self):
    #     for path in self.get_main_paths().values():
    #         if not os.path.exists(path):
    #             os.makedirs(path)

    # def get_image_paths(self):
    #     path = {
    #         "ORIGINAL_IMAGES_PATH" : ["workspace\datasets\original_images\Fresh_Ripe", "workspace\datasets\original_images\Fresh_Unripe", "workspace\datasets\original_images\Rotten_Ripe", "workspace\datasets\original_images\Rotten_Unripe"],
    #         "PREPROCESSED_IMAGES_PATH" : ["workspace\datasets\preprocessed_images\Fresh_Ripe", "workspace\datasets\preprocessed_images\Fresh_Unripe", "workspace\datasets\preprocessed_images\Rotten_Ripe", "workspace\datasets\preprocessed_images\Rotten_Unripe"],
    #     }

    #     return path

    # def set_image_paths(self):
    #     for path in self.get_image_paths().values():
    #         for label in path:
    #             if not os.path.exists(label):
    #                 os.makedirs(label)

    # def getFiles(self):
    #     files = {
    #         'PIPELINE_CONFIG':os.path.join(self.get_paths()['MODELS_PATH'], self.custom_name, 'pipeline.config'),
    #         'TF_RECORD_SCRIPT':os.path.join(self.get_paths()['TFRECORD_GENERATOR_PATH'], self.tfrecord_name),
    #         'LABEL_MAP':os.path.join(self.get_paths()['ANOTATIONS_PATH'], self.map_name),
    #     }
    #     return files

In [5]:
class FCLB_Datasets:
    #class' constructor
    def __init__(self, setup):
        self.setup = setup
    
    #acquiring uncategorized dataset
    def get_original_dataset(self, num1, num2, label):
        images_path = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        path = os.path.join(images_path, label)

        files = ["{}".format(i) for i in os.listdir(path)[num1:num2]]

        return files

    def copy_image_file(self, num1, num2, srcPath, dstPath):

        for label in self.setup.LABELS:
            for file in self.get_original_dataset(num1, num2, label):
                dst = os.path.join(dstPath, label, file)
                src = os.path.join(srcPath, label, file)
                shutil.copyfile(src, dst)

    #I call it segment function, wala na koy mahunahunaan
    def segment(self):
        original_dir = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        target_dir = self.setup.getWorkspaces()['PREPROCESSED_IMG']
        labels = self.setup.LABELS

        for label in labels:
            original = os.path.join(original_dir, label)
            target = os.path.join(target_dir, label)
            for img in os.listdir(original):
                self.segmentation(original, target, img, label)
 

    #this function used to segment an image file, it actually resize image, add contrast and lightness
    #and save it to the target directory
    #the purpose of uuid module is to give unique id or name to an image file
    def segmentation(self, base_dir, target_dir, fname, label):
        image = Image.open(os.path.join(base_dir, fname))
        size = (400,400)
        image.thumbnail(size)

        contrast = ImageEnhance.Contrast(image)
        contrast.enhance(1.8).save(os.path.join(target_dir, label + "-"+ '{}.jpg'.format(uuid.uuid1())))

        brightness = ImageEnhance.Brightness(image)
        brightness.enhance(1.2).save(os.path.join(target_dir, label +"-"+ '{}.jpg'.format(uuid.uuid1())))

    #compressed file for github upload
    def compressed(self):
        path = self.getDir('PREPROCESSED_IMAGES_PATH')
        sub_paths = self.getSubDirectories(path)
        ARCHIVE_PATH = os.path.join(path, 'preprocessed_images.tar.gz')
        tar = tarfile.open(ARCHIVE_PATH, "w:gz")
        for sub_path in sub_paths:
            dir = os.path.join(path, sub_path)
            tar.add(dir)
        tar.close()

# Preparing Workspace

Creating a workspace directory

In [6]:
setup = Setup_Path()
workspaces = setup.getWorkspaces()
for workspace in workspaces.values():
  if not os.path.exists(workspace):
    os.makedirs(workspace)

Create label directory for original images and preprocessed images

In [ ]:
for label in setup.LABELS:
  preprocessedImg_labelPath = os.path.join(setup.getWorkspaces()['PREPROCESSED_IMG'], label)
  if not os.path.exists(preprocessedImg_labelPath):
    os.makedirs(preprocessedImg_labelPath)

segment images and copy to preprocessed directory

In [ ]:
dataset = FCLB_Datasets(setup)
dataset.segment()

# Image Labelling

In [ ]:
labelImg = setup.getWorkspaces()['LABEL_IMG_PATH']
!cd {labelImg} && git clone https://github.com/tzutalin/labelImg.git

Cloning into 'labelImg'...
remote: Enumerating objects: 1846, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1846 (delta 37), reused 55 (delta 18), pack-reused 1759
Receiving objects: 100% (1846/1846), 232.81 MiB | 41.39 MiB/s, done.
Resolving deltas: 100% (1084/1084), done.


In [ ]:
labelImg = os.path.join(setup.getWorkspaces()['LABEL_IMG_PATH'], "labelImg")
!cd {labelImg} && pip install pyqt5
!cd {labelImg} && pyrcc5 -o libs/resources.py resources.qrc

# Partition Dataset

In [ ]:
url = "https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/d0e545609c5f7f49f39abc7b6a38cec3/partition_dataset.py"
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/partition_dataset.py'

In [ ]:
images_path = setup.getWorkspaces()['IMAGES_PATH']
!cd {dst} python partition_dataset.py -x -i {images_path} -r 0.1

/bin/bash: line 0: cd: too many arguments


# Download Tensorflow Object Detection API

In [7]:
tfod_api = setup.getWorkspaces()['TFOD_API_PATH']
!cd {tfod_api} && git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 61038, done.
remote: Total 61038 (delta 0), reused 0 (delta 0), pack-reused 61038
Receiving objects: 100% (61038/61038), 573.94 MiB | 31.68 MiB/s, done.
Resolving deltas: 100% (42513/42513), done.


Install Object Detection

In [37]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
src = wget.download(url)
protos_path = setup.getWorkspaces()['PROTOS_PATH']
shutil.move(src, protos_path)

Error: ignored

In [45]:
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(protos_path, 'bin'))   
!cd {research_path} && protoc object_detection/protos/*.proto --python_out=. && cp object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd {slim_path} && pip install -e . 

cp: cannot stat 'object_detection\packages\tf2\setup.py': No such file or directory
Obtaining file:///content/Tensorflow/models/research/slim
  Attempting uninstall: slim
    Found existing installation: slim 0.1
    Can't uninstall 'slim'. No files were found to uninstall.
  Running setup.py develop for slim


In [46]:
# !cd {research_path} && python object_detection/builders/model_builder_tf2_test.py
VERIFICATION_SCRIPT = os.path.join(research_path, 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Traceback (most recent call last):
  File "Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py", line 25, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


# Create Label Map

In [ ]:
labels = [{'name':'fresh', 'id':1}, {'name':'rotten', 'id':2}]

with open(setup.getFiles()['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create Tensorflow Records

In [ ]:
url = 'https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py'
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/generate_tfrecord.py'

In [ ]:
# Create train data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TRAINING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

# Create test data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TESTING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

Traceback (most recent call last):
  File "Tensorflow/scripts/preprocessing/generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
ModuleNotFoundError: No module named 'object_detection'
Traceback (most recent call last):
  File "Tensorflow/scripts/preprocessing/generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
ModuleNotFoundError: No module named 'object_detection'
